In [144]:
!pip install sklearn-crfsuite

You should consider upgrading via the 'c:\users\martin\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.
ERROR: Could not find a version that satisfies the requirement sklearn-grid_search (from versions: none)
ERROR: No matching distribution found for sklearn-grid_search
You should consider upgrading via the 'c:\users\martin\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [169]:
pip install --user -U scikit-learn==0.23.2

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Martin\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import spacy
import sklearn_crfsuite
# import math

from sklearn_crfsuite import metrics
from sklearn.metrics import classification_report
# from sklearn.preprocessing import MultiLabelBinarizer

import scipy.stats
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
# from sklearn.model_selection import GridSearchCV

# Helpers

In [12]:
nlp = spacy.load("en_core_web_sm")
WINDOW = 7

def build_dataframe(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            data.append(line.strip().split('\t'))
    df = pd.DataFrame(data)

    df = df.rename(columns={0: 'file', 1: 'sentence', 2: 'position', 3: 'token', 4: 'negCue'})

    #drop empty rows
    df = df.drop(df[(df.file == '')].index)

    df.insert(len(df.columns)-1, 'lemma', None)
    df.insert(len(df.columns)-1, 'postag', None)

    for i in range(1,(WINDOW+1)):
        df.insert(len(df.columns)-1, f'lemma_a{i}', None)  #lemma of token 1-position AFTER  token
        df.insert(len(df.columns)-1, f'lemma_b{i}', None)  #lemma of token 1-position BEFORE  token

        df.insert(len(df.columns)-1, f'pos_a{i}', None)    #postag of token 1-position AFTER  token
        df.insert(len(df.columns)-1, f'pos_b{i}', None)    #postag of token 1-position BEFORE  token

    df.insert(len(df.columns)-1, 'known_cue', None)        #if token was seen as a cue during training

    df = df.astype({'sentence': 'int32'})
    df = df.astype({'position': 'int32'})

    #Encode golden label
    negCue_dict = {"negCue":     {"O": '0', "B-NEG": '1', "I-NEG": '2'}}
    df = df.replace(negCue_dict)    

    return df

def feature_extraction(dataframe,known_cues=[]):

    column_names = dataframe.columns
    new_df = pd.DataFrame(columns=column_names)

    for _file in dataframe.file.unique():
        file_df = dataframe[dataframe['file']==_file]

        file_df = file_df.groupby(['sentence']).apply(applySentenceGroupBy)

        new_df = new_df.append(file_df)

    dataframe = new_df

    dataframe = set_known_cue_feature(dataframe,known_cues)

    return dataframe

def set_known_cue_feature(df,cues):
    new_df = pd.DataFrame(columns=df.columns)
    
    for i,row in df.iterrows():
        row['known_cue'] = True if row['token'] in cues else False
        new_df = new_df.append(row,ignore_index=True)

    return new_df
            
def applySentenceGroupBy(sentence_df):
    
    tokens = []
    for i,row in sentence_df.iterrows():
        tokens.append(row['token'])
    
    #reconstruct sentence from original df
    string = ' '.join([token for token in tokens])
    
    #use spacy to tokenize and extract info from sentence
    spacy_tokenized = nlp(string)
    
    sentence_df = add_token_features(sentence_df,spacy_tokenized,offset=1)    
    
    return sentence_df    

def add_token_features(sentence_df,spacy_tokenized,offset=1):
    i=0
    # tokens = [token for token in spacy_tokenized]
    
    sentence_ = pd.DataFrame(columns=sentence_df.columns)
    for idx,row in sentence_df.iterrows():
        
        # Both tokenizarions match:
        if row['token'] == spacy_tokenized[i].text:

            row = set_lemma_and_pos(row,spacy_tokenized,i)
        else:
            
            if row['token'] == spacy_tokenized[i-offset].text:
                row = set_lemma_and_pos(row,spacy_tokenized,i-offset)
            elif row['token'] == spacy_tokenized[i+offset].text:
                row = set_lemma_and_pos(row,spacy_tokenized,i+offset)
            else:
                #try one position more
                
                if row['token'] == spacy_tokenized[i-offset+1].text:
                    row = set_lemma_and_pos(row,spacy_tokenized,i-offset+1)
                elif row['token'] == spacy_tokenized[i+offset+1].text:
                    row = set_lemma_and_pos(row,spacy_tokenized,i+offset+1)                

        sentence_ = sentence_.append(row,ignore_index=True)
        i += 1
    
    return sentence_

def set_lemma_and_pos(row,spacy_tokenized,idx):
    #Lemma
    row['lemma']          = spacy_tokenized[idx].lemma_
    row['postag']         = spacy_tokenized[idx].tag_

    for i in range (1,(WINDOW+1)):
        row[f'lemma_b{i}']       = spacy_tokenized[idx-i].lemma_ if (idx + (1-i)) > 0 else None 
        row[f'lemma_a{i}']       = spacy_tokenized[idx+i].lemma_ if (idx+i) < len(spacy_tokenized) else None

        row[f'pos_b{i}']       = spacy_tokenized[idx-i].tag_ if (idx + (1-i)) > 0 else None 
        row[f'pos_a{i}']       = spacy_tokenized[idx+i].tag_ if (idx+i) < len(spacy_tokenized) else None

    return row

def get_training_cues(train_df):
    # print(train_df.head())
    cues= train_df.loc[(train_df['negCue']=='1')| (train_df['negCue']==1)]['token'].tolist()

    cues = list(set(cues)) #remove duplicates

    return cues

def dataframe2features(dataframe,known_cues=True,window=7):
    tokens = []
    labels = []
    for sentence_id, group in dataframe.groupby(['file','sentence']): #TODO arreglar esto para cuando saque el BREAK de files
        token = []
        label = []
        for _, row in group.iterrows():
            token_dict = {'token': row['token'],
                            'file': row['file'],
                            'sentence': row['sentence'],
                            'position': row['position']
                        }
            if known_cues:
                token_dict['known_cue'] = row['known_cue']
            
            if row['lemma'] != None:
                token_dict['lemma'] = row['lemma']
            if row['postag'] != None:
                token_dict['postag'] = row['postag']
            
            for i in range(1,window+1):
                if str(row[f'lemma_a{i}']) != 'nan':
                    token_dict[f'lemma_a{i}'] = row[f'lemma_a{i}']
                if str(row[f'lemma_b{i}']) != 'nan':
                    token_dict[f'lemma_b{i}'] = row[f'lemma_b{i}']                    
                if str(row[f'pos_b{i}']) != 'nan':
                    token_dict[f'pos_b{i}'] = row[f'pos_b{i}']                                        
                if str(row[f'pos_a{i}']) != 'nan':
                    token_dict[f'pos_a{i}'] = row[f'pos_a{i}']                                                            

            # if row['negCue'] != None:
            #     token_dict['negCue'] = row['negCue']


            token.append(token_dict)
            label.append(str(row['negCue']))
        tokens.append(token)
        labels.append(label)

    return tokens,labels

def train_crf_and_predict(X_train,y_train,X_test,y_test):
    crf = sklearn_crfsuite.CRF(
        algorithm='lbfgs',
        c1=0.1,
        c2=0.1,
        max_iterations=100,
        all_possible_transitions=True
    )

    crf.fit(X_train, y_train)

    labels = list(crf.classes_)
    sorted_labels = sorted(labels, key=lambda name: (name[1:], name[0]))

    y_pred = crf.predict(X_test)
    y_pred_ = []

    for sentence_preds in y_pred:
        y_pred_.extend(sentence_preds)

    y_test = y_test.astype({'negCue': str})

    report = classification_report(y_test, y_pred_, target_names=sorted_labels)
    print(report)


    return crf,report


# Pre-Processing and Feature Engineering (TRAIN AND TEST DFs)

## Preprocess & save

In [49]:
train_df = build_dataframe("corpus/SEM-2012-SharedTask-CD-SCO-training-simple.v2.txt")
test_df = build_dataframe("corpus/SEM-2012-SharedTask-CD-SCO-dev-simple.v2.txt")

known_cues = get_training_cues(train_df)

train_df = feature_extraction(train_df,known_cues)
test_df = feature_extraction(test_df,known_cues)

train_df.to_csv('corpus/train_df_preproc.csv', sep='\t',index=False)
test_df.to_csv('corpus/test_df_preproc.csv', sep='\t',index=False)

train_df = pd.read_csv('corpus/train_df_preproc.csv', sep='\t')
test_df = pd.read_csv('corpus/test_df_preproc.csv', sep='\t')

train_df

,file,sentence,position,token,lemma,postag,lemma_a1,lemma_b1,pos_a1,pos_b1,...,lemma_a6,lemma_b6,pos_a6,pos_b6,lemma_a7,lemma_b7,pos_a7,pos_b7,known_cue,negCue
0,baskervilles01,0,0,Chapter,chapter,NN,1,NaN,CD,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0
1,baskervilles01,0,1,1.,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0
2,baskervilles01,0,2,Mr.,Mr.,NNP,Sherlock,.,NNP,.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0
3,baskervilles01,0,3,Sherlock,Sherlock,NNP,Holmes,Mr.,NNP,NNP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0
4,baskervilles01,0,4,Holmes,Holmes,NNP,NaN,Sherlock,NaN,NNP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65446,baskervilles14,270,58,slopes,slope,NNS,of,russet,IN,NN,...,NaN,until,NaN,IN,NaN,away,NaN,RB,False,0
65447,baskervilles14,270,59,of,of,IN,the,slope,DT,NNS,...,NaN,it,NaN,PRP,NaN,until,NaN,IN,False,0
65448,baskervilles14,270,60,the,the,DT,moor,of,NN,IN,...,NaN,merge,NaN,VBD,NaN,it,NaN,PRP,False,0
65449,baskervilles14,270,61,moor,moor,NN,.,the,.,DT,...,NaN,into,NaN,IN,NaN,merge,NaN,VBD,False,0


## Read preprocessed DFs

In [4]:
train_df = pd.read_csv('corpus/train_df_preproc.csv', sep='\t')
test_df = pd.read_csv('corpus/test_df_preproc.csv', sep='\t')

train_df

,file,sentence,position,token,lemma,postag,lemma_a1,lemma_b1,pos_a1,pos_b1,...,lemma_a6,lemma_b6,pos_a6,pos_b6,lemma_a7,lemma_b7,pos_a7,pos_b7,known_cue,negCue
0,baskervilles01,0,0,Chapter,chapter,NN,1,NaN,CD,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0
1,baskervilles01,0,1,1.,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0
2,baskervilles01,0,2,Mr.,Mr.,NNP,Sherlock,.,NNP,.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0
3,baskervilles01,0,3,Sherlock,Sherlock,NNP,Holmes,Mr.,NNP,NNP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0
4,baskervilles01,0,4,Holmes,Holmes,NNP,NaN,Sherlock,NaN,NNP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65446,baskervilles14,270,58,slopes,slope,NNS,of,russet,IN,NN,...,NaN,until,NaN,IN,NaN,away,NaN,RB,False,0
65447,baskervilles14,270,59,of,of,IN,the,slope,DT,NNS,...,NaN,it,NaN,PRP,NaN,until,NaN,IN,False,0
65448,baskervilles14,270,60,the,the,DT,moor,of,NN,IN,...,NaN,merge,NaN,VBD,NaN,it,NaN,PRP,False,0
65449,baskervilles14,270,61,moor,moor,NN,.,the,.,DT,...,NaN,into,NaN,IN,NaN,merge,NaN,VBD,False,0


## Training and Predicting with different set of features

### Window = 7 and Known Cues

In [15]:
print("#######################################################################################")
print("WINDOW = 7, KNOWN_CUES = TRUE")

train_df_w7_kc = train_df 
test_df_w7_kc  = test_df 

X_train_w7_kc, y_train = dataframe2features(train_df_w7_kc)
X_test_w7_kc,  y_test  = dataframe2features(test_df_w7_kc)

crf_w7_kc, report_w7_kc = train_crf_and_predict(X_train_w7_kc,y_train,X_test_w7_kc,test_df_w7_kc['negCue'])

print("#######################################################################################")
print("WINDOW = 7, KNOWN_CUES = FALSE")

train_df_w7 = train_df.drop(columns=['known_cue'])
test_df_w7  = test_df.drop(columns=['known_cue'])

X_train_w7, y_train = dataframe2features(train_df_w7,known_cues=False)
X_test_w7,  y_test  = dataframe2features(test_df_w7,known_cues=False)

crf_w7, report_w7 = train_crf_and_predict(X_train_w7,y_train,X_test_w7,test_df_w7['negCue'])

print("#######################################################################################")
print("WINDOW = 6, KNOWN_CUES = TRUE")

train_df_w6_kc = train_df.drop(columns=['lemma_a7','lemma_b7','pos_a7','pos_b7']) 
test_df_w6_kc  = test_df.drop(columns=['lemma_a7','lemma_b7','pos_a7','pos_b7']) 

X_train_w6_kc, y_train = dataframe2features(train_df_w6_kc,window=6)
X_test_w6_kc,  y_test  = dataframe2features(test_df_w6_kc,window=6)

crf_w6_kc, report_w6_kc = train_crf_and_predict(X_train_w6_kc,y_train,X_test_w6_kc,test_df_w6_kc['negCue'])

print("#######################################################################################")
print("WINDOW = 6, KNOWN_CUES = FALSE")

train_df_w6 = train_df.drop(columns=['lemma_a7','lemma_b7','pos_a7','pos_b7','known_cue'])
test_df_w6  = test_df.drop(columns=['lemma_a7','lemma_b7','pos_a7','pos_b7','known_cue'])

X_train_w6, y_train = dataframe2features(train_df_w6,known_cues=False,window=6)
X_test_w6,  y_test  = dataframe2features(test_df_w6,known_cues=False,window=6)

crf_w6, report_w6 = train_crf_and_predict(X_train_w6,y_train,X_test_w6,test_df_w6['negCue'])

print("#######################################################################################")
print("WINDOW = 5, KNOWN_CUES = TRUE")

train_df_w5_kc = train_df.drop(columns=['lemma_a7','lemma_b7','pos_a7','pos_b7','lemma_a6','lemma_b6','pos_a6','pos_b6']) 
test_df_w5_kc  = test_df.drop(columns=['lemma_a7','lemma_b7','pos_a7','pos_b7','lemma_a6','lemma_b6','pos_a6','pos_b6']) 

X_train_w5_kc, y_train = dataframe2features(train_df_w5_kc,window=5)
X_test_w5_kc,  y_test  = dataframe2features(test_df_w5_kc,window=5)

crf_w5_kc, report_w5_kc = train_crf_and_predict(X_train_w5_kc,y_train,X_test_w5_kc,test_df_w5_kc['negCue'])

print("#######################################################################################")
print("WINDOW = 5, KNOWN_CUES = FALSE")

train_df_w5 = train_df.drop(columns=['lemma_a7','lemma_b7','pos_a7','pos_b7','known_cue','lemma_a6','lemma_b6','pos_a6','pos_b6'])
test_df_w5  = test_df.drop(columns=['lemma_a7','lemma_b7','pos_a7','pos_b7','known_cue','lemma_a6','lemma_b6','pos_a6','pos_b6'])

X_train_w5, y_train = dataframe2features(train_df_w5,known_cues=False,window=5)
X_test_w5,  y_test  = dataframe2features(test_df_w5,known_cues=False,window=5)

crf_w5, report_w5 = train_crf_and_predict(X_train_w5,y_train,X_test_w5,test_df_w5['negCue'])


#######################################################################################
WINDOW = 7, KNOWN_CUES = TRUE
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     13388
           1       0.96      0.87      0.91       176
           2       1.00      0.67      0.80         3

    accuracy                           1.00     13567
   macro avg       0.98      0.85      0.90     13567
weighted avg       1.00      1.00      1.00     13567

#######################################################################################
WINDOW = 7, KNOWN_CUES = FALSE
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     13388
           1       0.95      0.78      0.86       176
           2       1.00      0.67      0.80         3

    accuracy                           1.00     13567
   macro avg       0.98      0.81      0.88     13567
weighted avg       1.00      1.00      1.00     13567

#

In [11]:
asd = test_df_w7_kc['negCue']

asd = asd.astype({'negCue': str})
asd

0        0
1        0
2        0
3        0
4        0
        ..
13562    0
13563    0
13564    0
13565    0
13566    0
Name: negCue, Length: 13567, dtype: object

### Window = 7 and NOT Known Cues

In [40]:
train_df_w7 = train_df.drop(columns=['known_cue'])
test_df_w7  = test_df.drop(columns=['known_cue'])

X_train_w7, y_train = dataframe2features(train_df_w7,known_cues=False)
X_test_w7,  y_test  = dataframe2features(test_df_w7,known_cues=False)

crf_w7, report_w7 = train_crf_and_predict(X_train_w7,y_train,X_test_w7,test_df_w7['negCue'])


#### Training Finished! #######
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     13388
           1       0.95      0.78      0.86       176
           2       1.00      0.67      0.80         3

    accuracy                           1.00     13567
   macro avg       0.98      0.81      0.88     13567
weighted avg       1.00      1.00      1.00     13567



C:\Users\Martin\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


(CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
     keep_tempfiles=None, max_iterations=100),
 '              precision    recall  f1-score   support\n\n           0       1.00      1.00      1.00     13388\n           1       0.95      0.78      0.86       176\n           2       1.00      0.67      0.80         3\n\n    accuracy                           1.00     13567\n   macro avg       0.98      0.81      0.88     13567\nweighted avg       1.00      1.00      1.00     13567\n')

UnboundLocalError: local variable 'test_df' referenced before assignment

In [36]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)

crf.fit(X_train_w7, y_train)

labels = list(crf.classes_)
sorted_labels = sorted(labels, key=lambda name: (name[1:], name[0]))

y_pred = crf.predict(X_test_w7)
y_pred_ = []

for sentence_preds in y_pred:
    y_pred_.extend(sentence_preds)

test_df['pred'] = y_pred_
test_df = test_df.astype({"negCue": str})


print(classification_report(test_df_w7['negCue'], y_pred_ , target_names=sorted_labels))
# print(classification_report(test_df['negCue'], test_df['pred']  , target_names=sorted_labels))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     13388
           1       0.95      0.78      0.86       176
           2       1.00      0.67      0.80         3

    accuracy                           1.00     13567
   macro avg       0.98      0.81      0.88     13567
weighted avg       1.00      1.00      1.00     13567



In [26]:
#################################################################################

In [9]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)

crf.fit(X_train, y_train)


C:\Users\Martin\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [10]:
labels = list(crf.classes_)
sorted_labels = sorted(labels, key=lambda name: (name[1:], name[0]))

y_pred = crf.predict(X_test)
y_pred_ = []

for sentence_preds in y_pred:
    y_pred_.extend(sentence_preds)

test_df['pred'] = y_pred_
test_df = test_df.astype({"negCue": str})


print(classification_report(test_df['negCue'], test_df['pred'], target_names=sorted_labels))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     13388
           1       0.96      0.87      0.91       176
           2       1.00      0.67      0.80         3

    accuracy                           1.00     13567
   macro avg       0.98      0.85      0.90     13567
weighted avg       1.00      1.00      1.00     13567



## Hyper parameter optimization

In [11]:
# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)

rs.fit(X_train, y_train)


print('best params:', rs.best_params_)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  5.6min finished


best params: {'c1': 1.1210252392427602, 'c2': 0.031129825505882648}


################


In [14]:
crf_hyper = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=1.121,
    c2=0.031,
    max_iterations=100,
    all_possible_transitions=True
)

crf_hyper.fit(X_train, y_train)


labels = list(crf_hyper.classes_)

y_pred = crf_hyper.predict(X_test)
# y_pred_ = MultiLabelBinarizer().fit_transform(y_pred)

y_pred_ = []
for sentence_preds in y_pred:
    y_pred_.extend(sentence_preds)

test_df['pred'] = y_pred_
test_df = test_df.astype({"negCue": str})

# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)

print(classification_report(test_df['negCue'], test_df['pred'], target_names=sorted_labels))

KeyboardInterrupt: 

In [18]:
train_df

,file,sentence,position,token,lemma,postag,lemma_a1,lemma_b1,pos_a1,pos_b1,...,lemma_a6,lemma_b6,pos_a6,pos_b6,lemma_a7,lemma_b7,pos_a7,pos_b7,known_cue,negCue
0,baskervilles01,0,0,Chapter,chapter,NN,1,NaN,CD,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0
1,baskervilles01,0,1,1.,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0
2,baskervilles01,0,2,Mr.,Mr.,NNP,Sherlock,.,NNP,.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0
3,baskervilles01,0,3,Sherlock,Sherlock,NNP,Holmes,Mr.,NNP,NNP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0
4,baskervilles01,0,4,Holmes,Holmes,NNP,NaN,Sherlock,NaN,NNP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65446,baskervilles14,270,58,slopes,slope,NNS,of,russet,IN,NN,...,NaN,until,NaN,IN,NaN,away,NaN,RB,False,0
65447,baskervilles14,270,59,of,of,IN,the,slope,DT,NNS,...,NaN,it,NaN,PRP,NaN,until,NaN,IN,False,0
65448,baskervilles14,270,60,the,the,DT,moor,of,NN,IN,...,NaN,merge,NaN,VBD,NaN,it,NaN,PRP,False,0
65449,baskervilles14,270,61,moor,moor,NN,.,the,.,DT,...,NaN,into,NaN,IN,NaN,merge,NaN,VBD,False,0


In [1]:
train_df

NameError: name 'train_df' is not defined